# Score Scenarios

Run the cells below to score the configured scenarios. Any unscored scenarios will be scored.

To re-score a scenario, open its corresponding file geodatabase and delete its `scores` and `scores_summary` table.

In [ ]:
import sys
import os
import arcpy

import pandas as pd
from arcgis.features import SpatialDataFrame

src = os.path.join(os.path.abspath("."), 'src')
if src not in sys.path:
    sys.path.append(src)
    
from ato_tools import ato

if 'ato_tools' in sys.modules:
    import importlib
    importlib.reload(ato)

base_path = os.path.abspath(".")

baseline_gdb = "baseline.gdb"

In [ ]:
modes = list()
for file in os.listdir(r'scenario'):
    d = os.path.join(base_path, r'scenario', file)
    if os.path.isdir(d) and not d.endswith('.gdb'):
        modes.append(file)

to_score = list()
for mode in ['Cycling', 'Driving', 'Transit']:
    scenario_folder = os.path.join('scenario', mode)
    for file in os.listdir(scenario_folder):
        d = os.path.join(base_path, scenario_folder, file)
        if os.path.isdir(d) and d.endswith('.gdb'):
            if arcpy.Exists(os.path.join(d, 'ato')) == False:
                print(d)
                to_score.append((d, mode))

In [ ]:
# score unscored networks
test = False

# if testing, use a subset of centroids
if test:
    centroids = r"baseline.gdb\taz_centroids_sample"
else:
    centroids = r'baseline.gdb\taz_centroids'

for scenario in to_score:
    gdb, mode = scenario
    ato.skim(
        gdb = gdb,
        mode = mode,
        centroids = centroids,
        out_table = os.path.join(gdb, r"skim_matrix")
    )
    ato.score(
        gdb = gdb,
        taz_table = r'baseline.gdb\taz_table'
    )

If the loop above fails due to an invalid network, you can rebuild the network by running `ato.build()` on the full network dataset path, e.g. `ato.build(r'scenario\Transit\ogden_local_bus.gdb\NetworkDataset\NetworkDataset_ND')`

In [ ]:
#score land use projects
to_score = list()
scenario_folder = os.path.join('scenario', 'land_use')
for file in os.listdir(scenario_folder):
    d = os.path.join(base_path, scenario_folder, file)
    if os.path.isdir(d) and d.endswith('.gdb'):
        if arcpy.Exists(os.path.join(d, 'ato')) == False:
            
            to_score.append(d)

for gdb in to_score:
    print('Scoring {0}'.format(gdb))
    
    ato.score(
        gdb = gdb,
        skim_matrix = os.path.join(baseline_gdb, r"skim_driving"), 
        job_per_hh = 1.808755,
        out_table = os.path.join(gdb, r"ato_driving")
    )
    
    ato.score(
        gdb = gdb,
        skim_matrix = os.path.join(baseline_gdb, r"skim_cycling"), 
        job_per_hh = 1.808755,
        out_table = os.path.join(gdb, r"ato_cycling")
    )
        
    ato.score(
        gdb = gdb,
        skim_matrix = os.path.join(baseline_gdb, r"skim_transit"), 
        job_per_hh = 1.808755,
        out_table = os.path.join(gdb, r"ato_transit")
    )
    

## Tabulate Scores Across Scenarios

The cells below read in the scores from all scored scenarios and combined into a single summary table (`scenario\scenario_scores.csv`)

Scores for individual projects can be extracted from the `scores_summary` table within each file geodatabase.

In [ ]:
scenarios = list()

for mode in ['Driving', 'Transit', 'Cycling', 'Land_Use']:
    scenario_folder = os.path.join('scenario', mode)
    for file in os.listdir(scenario_folder):
        d = os.path.join(scenario_folder, file)
        if os.path.isdir(d) and d.endswith('.gdb'):
            if arcpy.Exists(os.path.join(d, 'ato')) == True:
                scenarios.append({
                                "name": file[:-4],
                                "mode": mode,
                                "gdb_path": d
                            })

In [ ]:
land_use_scenario_scores = pd.DataFrame(columns = 
    ['Name', 'driving_comp', 'cycling_comp', 'transit_comp', 
     'cycling_to_auto', 'transit_to_auto']
)

land_use_scenarios = filter(lambda x: x['mode'] == 'Land_Use', scenarios)

for scenario in land_use_scenarios:
    scores = {}

    for mode in ['driving', 'transit', 'cycling']:
        baseline = os.path.join('baseline.gdb', 'ato_' + mode)
        scenario_scores = os.path.join(scenario['gdb_path'], 'ato_' + mode)

        scores[mode] = ato.diff(baseline, scenario_scores)

    vals = {
        "Name": scenario['name'],
        'driving_comp': scores['driving'],
        'cycling_comp': scores['cycling'],
        'transit_comp': scores['transit'], 
        'cycling_to_auto': round(scores['cycling'] / scores['driving'], 2),
        'transit_to_auto': round(scores['transit'] / scores['driving'], 2)
    }
    
    land_use_scenario_scores = land_use_scenario_scores.append(vals, ignore_index=True)

In [ ]:
land_use_scenario_scores

In [ ]:
scenario_scores = pd.DataFrame(columns = 
    ['Name', 'Mode', 'hh_access', 'jobs_access', 'comp_access',
     'pov_accessible_jobs', 'minority_accessible_jobs', 
     'zero_car_accessible_jobs', 'efa_accessible_jobs']
)

equity_taz = pd.read_csv('equity_taz.csv')

for scenario in filter(lambda x: x['mode'] != 'Land_Use', scenarios):
    
    mode = scenario['mode']
    
    if mode == 'Driving':
        baseline = r'baseline.gdb\ato_driving'
    elif mode == 'Transit':
        baseline = r'baseline.gdb\ato_transit'
    elif mode == 'Cycling':
        baseline = r'baseline.gdb\ato_cycling'
    elif mode == 'Land_Use':
        baseline = r'baseline.gdb\ato_comp'
    
    scores_table = os.path.join(scenario['gdb_path'], 'scores')
    
    if not arcpy.Exists(scores_table):
        ato.diff(
            baseline = baseline,
            scenario = os.path.join(scenario['gdb_path'], 'ato'),
            out_table = scores_table
        )
    
    df = pd.DataFrame(arcpy.da.TableToNumPyArray(scores_table, '*'))

    df = pd.merge(
        df, 
        equity_taz, 
        on='CO_TAZID', 
        how="left"
    )

    vals = {
        "Name": scenario['name'],
        'Mode': mode,
        "hh_access": df['diff_hh'].sum(),
        "jobs_access": df['diff_jobs'].sum(),
        "comp_access": df['diff_ato'].sum(),
        'pov_accessible_jobs': (df['diff_jobs'] * df['SD_Pov']).sum(),
        'minority_accessible_jobs': (df['diff_jobs'] * df['SD_Minorit']).sum(),
        'zero_car_accessible_jobs': (df['diff_jobs'] * df['SD_ZeroCar']).sum(),
        'efa_accessible_jobs': (df['diff_jobs'] * df[['SD_Pov', 'SD_Minorit', 'SD_ZeroCar']].max(axis=1)).sum()
    }
    
    scenario_scores = scenario_scores.append(vals, ignore_index=True)

In [ ]:
scenario_scores

In [ ]:
land_use_scenario_scores.to_csv(r'scenario\land_use_scenario_scores.csv')
scenario_scores.to_csv(r'scenario\scenario_scores.csv')